In [ ]:
# Import essential libraries
import pandas as pd

%matplotlib inline

# Import other libraries
import os
import sys

# Import our own libraries
sys.path.append("../lib/")
import eyegaze as eg

In [ ]:
# 全参加者・全フェーズの定義
participants = {
    "A": ["P001", "P002", "P005", "P006", "P008", "P009", "P010", "P011", "P016", "P017"],
    "B": ["P003", "P004", "P007", "P012", "P013", "P014", "P015", "P018", "P019", "P020"],
}
phases = ["pre", "post", "training1", "training2", "training3"]

# 補正パラメータの読み込み（全参加者分を一括で読み込む）
corrections_path = "../../data/working/corrections/all_segment_corrections.csv"
all_corrections = pd.read_csv(corrections_path)
print(f"補正パラメータ: {len(all_corrections)}行")

# 処理対象の確認
total = sum(len(pids) for pids in participants.values()) * len(phases)
print(f"処理対象: {total}件（{sum(len(pids) for pids in participants.values())}名 × {len(phases)}フェーズ）")

In [ ]:
# 全参加者・全フェーズの一括処理（並列版）
from concurrent.futures import ProcessPoolExecutor, as_completed

# タスクリストの構築
corrections_path = "../../data/working/corrections/all_segment_corrections.csv"
tasks = []
for group_letter, participant_ids in participants.items():
    for participant_id in participant_ids:
        for phase in phases:
            tasks.append({
                'group_letter': group_letter,
                'participant_id': participant_id,
                'phase': phase,
                'corrections_path': os.path.abspath(corrections_path),
                'input_root': os.path.abspath('../../data/input'),
                'output_root': os.path.abspath('../../data/output'),
                'working_root': os.path.abspath('../../data/working'),
            })

total = len(tasks)
n_workers = min(os.cpu_count(), total)
print(f"Total: {total} tasks, Workers: {n_workers}")

# 並列実行
completed_count = 0
error_count = 0
errors = []

with ProcessPoolExecutor(max_workers=n_workers) as executor:
    futures = {executor.submit(eg._processOnePhaseWorker, t): t for t in tasks}
    for future in as_completed(futures):
        result = future.result()
        if result['success']:
            completed_count += 1
            print(f"  [{completed_count + error_count}/{total}] OK: {result['label']} ({result['n_segments']} segments)")
        else:
            error_count += 1
            errors.append((result['label'], result['error']))
            print(f"  [{completed_count + error_count}/{total}] ERROR: {result['label']} - {result['error'].splitlines()[0]}")

# --- 結果サマリー ---
print(f"\n{'='*60}")
print(f"完了: {completed_count}/{total}件 (エラー: {error_count}件)")
if errors:
    for label, msg in errors:
        print(f"  {label}: {msg}")